# Translate Messages for THR 17

Read the messages from src/Messages.ts and make sure they each have a translation into the requested target language.

I give preference first to the original contents, then to the given `po` file assuming a human did the work to create it, and finally to
Google Translate.

The idea is to create a new project on Transifex to get humans to help but to use the old THR translations and Google until then.

In [17]:
import Args # my args module
import polib
import re
import os.path as osp
import requests
import json
from apiclient.discovery import build

In [34]:
args = Args.Parse(
    po='',                      # existing po file if any
    src='src/Messages.ts',   # where to find the needed strings
    lang=str,                   # 2-letter iso language code
    messages=str,               # the json file to read/write with the translations
    key=str,                    # Google API key
    _config='./scripts/translatekey.json'
)
args

args: src=str messages=str key=str lang=str po=str


ValueError: missing required arguments {'messages', 'lang'}

In [11]:
# read the existing po file if any
po = {}
if args.po:
    pof = polib.pofile(args.po)
    for entry in pof:
        po[entry.msgid] = entry.msgstr

In [13]:
# extract strings from my Messages.ts file
msgpat = re.compile(r"\s+(\w+):\s*'(.*)',")
enMessages = []
for line in open(args.src, 'r'):
    m = msgpat.match(line)
    if m:
        enMessages.append(m.groups())

In [26]:
# read the existing translations
old = {}
if osp.exists(args.messages):
    old = json.load(open(args.messages, 'r'))

{'AllTopics': 'Alle Themen',
 'Alph': 'Alphabet',
 'AlternatePictureAndText': 'Alternatives Bild und Text',
 'Anim': 'Tiere und Natur',
 'AnyRating': 'alle Bewertungen',
 'ArtM': 'Kunst und Musik',
 'Audience': 'Leser',
 'Biog': 'Biografien',
 'ButtonSize': 'Knopfgröße',
 'Close': 'schließen',
 'EnterTextToSearch': 'Suchbegriff eingeben',
 'Fair': 'Märchen',
 'Fict': 'Fiktion',
 'FontSize': 'Schriftgröße',
 'Food': 'Lebensmittel',
 'Heal': 'Gesundheit',
 'Hist': 'Geschichte',
 'Holi': 'Urlaub',
 'IncludeUnreviewed': 'noch nicht überprüfte Bücher',
 'Language': 'Sprache',
 'Math': 'Mathematik und Naturwissenschaften',
 'Nurs': 'Kinderreime',
 'Peop': 'Menschen und Orte',
 'Poet': 'Dichtung',
 'RatedC': 'mit "V" bewertet / Vorsicht',
 'RatedE': 'mit "A" bewertet / Alle',
 'ReadingControls': 'Lesekontrollen',
 'Recr': 'Erholung und Freizeit',
 'ReviewStatus': 'Überprüfungsstatus',
 'ReviewedOnly': 'nur überprüfte Bücher anzeigen',
 'Search': 'suchen',
 'SearchFor': 'Suche nach',
 'Spor': 

In [21]:
API_KEY=args.key
def g_translate(message, lang):
    service = build('translate', 'v2', developerKey=API_KEY)
    request = service.translations().list(q=message, target=lang)
    response = request.execute()
    return response['translations'][0]['translatedText']

In [23]:
# Translate the ones we can't find elsewhere
new = {}
for key, message in enMessages:
    if key in old:
        new[key] = old[key]
    elif message in po:
        new[key] = po[message]
    else:
        new[key] = g_translate(message, args.lang)
        print(key, new[key])

home Startseite
find Bücher finden
choose Wähle ein Buch
ButtonSize Knopfgröße
normal Normal
medium Mittel
large Groß
ReadingControls Lesekontrollen
FontSize Schriftgröße
AlternatePictureAndText Alternatives Bild und Text
Voice Stimme


In [25]:
json.dump(obj=new, fp=open(args.messages, 'w'), indent=2, sort_keys=True)